In [15]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime
from decimal import Decimal, InvalidOperation
import base64
from datetime import datetime, timedelta
import pytz

pd.options.display.float_format = '{:,.14f}'.format

In [ ]:
# setup
load_dotenv()
TOKEN_URL = 'https://api.ticket360.com.br/auth/oauth/access_token'
API_URL = 'https://api.ticket360.com.br'
EVENT_ID = '31407'
EVENT_NAME = 'Camarote-samba-2026-inimigosHP-desfile-campea'
MAX_RETRIES = 3  #Max request until fail
TIMEOUT = 30  # Seconds
DATA_DIR = 'ticket_data'
CONSOLIDATED_FILE = os.path.join(DATA_DIR, "consolidated.json")

os.makedirs(DATA_DIR,exist_ok=True)

In [ ]:
def get_access_token():
    '''Get access token with error treatment'''
    for attempt in range(MAX_RETRIES):
        try:
            auth_string = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
            auth_base64 = base64.b64encode(auth_string.encode()).decode()
            
            response = requests.post(
                TOKEN_URL,
                headers={
                    "Authorization": f"Basic {auth_base64}",
                    "Content-Type": "application/x-www-form-urlencoded"
                },
                data={"grant_type": "client_credentials"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            return response.json().get("access_token")
        except requests.exceptions.Timeout:
            print(f"Timeout (tentativa {attempt + 1}/{MAX_RETRIES})")
            if attempt < MAX_RETRIES - 1:
                time.sleep(5)
        except Exception as e:
            print(f"Error to get token: {type(e).__name__} - {str(e)}")
            return None
    return None

In [ ]:
def fetch_report(token, start_date=None, end_date=None):
    '''Search data from API using pagination'''
    try:
        base_url = f"{API_URL}/sales/reports/consolidated/{EVENT_ID}?filter=status=paid&ticket.status=active"
        
        if start_date:
            base_url += f"&startDate={start_date}"
        if end_date:
            base_url += f"&endDate={end_date}"
        
        offset = 0
        limit = 1000
        all_sales = []
        
        while True:
            url = f"{base_url}&limit={limit}&offset={offset}"
            response = requests.get(
                url,
                headers={"Authorization": f"Bearer {token}"},
                timeout=TIMEOUT
            )
            response.raise_for_status()
            data = response.json()
            sales = data.get('sales', [])
            all_sales.extend(sales)
            
            if len(sales) < limit:
                break
                
            offset += limit
        
        return {'sales': all_sales}
    except Exception as e:
        print(f"Error to search data: {type(e).__name__} - {str(e)}")
        return None

In [19]:
def remove_today_data(df):
    hoje = datetime.now().date()

    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df[df['date'].dt.date != hoje]
    return df

In [ ]:
def save_to_json(data, filename):
    '''Save data into JSON file with treatment of date'''
    def convert_timestamps(obj):
        if isinstance(obj, pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, datetime):
            return obj.isoformat()
        return obj
    
    with open(filename, 'w') as f:
        if isinstance(data, pd.DataFrame):
            json_data = data.to_dict(orient='records')
            json.dump(json_data, f, indent=2, default=convert_timestamps)
        else:
            json.dump(data, f, indent=2, default=convert_timestamps)

In [ ]:
def consolidate_data(new_data_file):
    '''Make append with new data request'''
    # Carregar dados existentes
    if os.path.exists(CONSOLIDATED_FILE):
        try:
            df_existente = pd.read_json(CONSOLIDATED_FILE, lines=True)
        except:
            df_existente = pd.DataFrame()
    else:
        df_existente = pd.DataFrame()
    
    # Carregar novos dados
    df_novos = pd.read_json(new_data_file)
    
    # Combinar dados
    if df_existente.empty:
        df_final = df_novos
    else:
        df_final = pd.concat([df_existente, df_novos])
    
    # Salvar consolidado
    df_final.to_json(CONSOLIDATED_FILE, orient='records', lines=True)
    return df_final

In [ ]:
def main():
    print("Starting data collecting...")
    token = get_access_token()
    if not token:
        print("Fail at authentication")
        return
    
    # Coletar todos os dados
    report_data = fetch_report(token)
    if not report_data or 'sales' not in report_data:
        print("Fail at get historical data.")
        return
    
    # Converter para DataFrame
    df = pd.DataFrame(report_data['sales'])
    
    
    # Remover dados do dia atual
    df_clean = remove_today_data(df)
    
    
    # Salvar dados processados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    processed_file = os.path.join(DATA_DIR, f"processed_historical_{timestamp}_{EVENT_NAME}.json")
    save_to_json(df, processed_file)
    print(f"Dados históricos processados salvos em: {processed_file}")
    
    '''
    # Consolidar dados
    df_consolidated = consolidate_data(processed_file)
    print(f"Dados consolidados salvos em: {CONSOLIDATED_FILE} ({len(df_consolidated)} registros)")
    '''

if __name__ == "__main__":
    main()

Iniciando coleta histórica...
Dados históricos processados salvos em: ticket_data\processed_historical_20250825_094140_Camarote-samba-2026-inimigosHP-desfile-campea.json


In [23]:

def mostrar_datas_extremas(caminho_arquivo):
    # Abrir e ler o arquivo JSON
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)

    # Extrair as datas (assumindo que cada item tem a chave 'date')
    lista_datas = [item['date'] for item in dados]

    # Converter as strings para objetos datetime
    lista_datas_convertidas = [datetime.fromisoformat(d) for d in lista_datas]

    # Encontrar a mais antiga e a mais recente
    data_mais_antiga = min(lista_datas_convertidas)
    data_mais_recente = max(lista_datas_convertidas)

    # Mostrar no formato ISO (AAAA-MM-DD)
    print("Data mais antiga:", data_mais_antiga.date())
    print("Data mais recente:", data_mais_recente.date())

    #Transformar o arquivo JSON em um dataframe
    df = pd.read_json("ticket_data/processed_historical_20250825_091937_Camarote-samba-2026-inimigosHP-desfile-campea.json")
    contagem_registro = df['id'].count()
    print(f"Contagem de linhas: {contagem_registro}")

mostrar_datas_extremas("ticket_data/processed_historical_20250825_091937_Camarote-samba-2026-inimigosHP-desfile-campea.json")


Data mais antiga: 2025-07-30
Data mais recente: 2025-08-10
Contagem de linhas: 12


In [24]:
df_teste = pd.read_json("ticket_data/processed_historical_20250825_091937_Camarote-samba-2026-inimigosHP-desfile-campea.json")
display(df_teste.sort_values(by='date', ascending= True))

,id,module,date,event.id,event.name,event.venue.name,payment.method,payment.method.brand,payment.method.installments,delivery.method,...,ticket.transfer.receiver.gender,ticket.transfer.receiver.birthDate,ticket.transfer.receiver.address.district,ticket.transfer.receiver.address.city,ticket.transfer.receiver.address.state,ticket.entranceDate,ticket.status,virtualBoxOffice,geolocation,status
1,10510954,site,2025-07-30 11:40:37-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Master,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '147.161.128.170', 'country': 'BR', 're...",paid
2,10510954,site,2025-07-30 11:40:37-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Master,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '147.161.128.170', 'country': 'BR', 're...",paid
3,10510974,mobile,2025-07-30 11:50:43-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '177.60.83.134', 'country': 'BR', 'regi...",paid
4,10510974,mobile,2025-07-30 11:50:43-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '177.60.83.134', 'country': 'BR', 'regi...",paid
0,10512905,mobile,2025-07-31 11:06:07-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '177.26.236.191', 'country': 'BR', 'reg...",paid
5,10512905,mobile,2025-07-31 11:06:07-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '177.26.236.191', 'country': 'BR', 'reg...",paid
6,10513622,site,2025-07-31 15:49:50-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '187.182.238.83', 'country': 'BR', 'reg...",paid
7,10513622,site,2025-07-31 15:49:50-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '187.182.238.83', 'country': 'BR', 'reg...",paid
8,10513622,site,2025-07-31 15:49:50-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '187.182.238.83', 'country': 'BR', 'reg...",paid
9,10513622,site,2025-07-31 15:49:50-03:00,31407,Camarote Samba 2026 com Inimigos da HP Desfile...,Sambódromo do Anhembi,credit-card,Visa,3,virtual,...,NaN,NaN,NaN,NaN,NaN,NaN,active,False,"{'ip': '187.182.238.83', 'country': 'BR', 'reg...",paid
